In [1]:
import pandas as pd
from pandas import DataFrame
from datetime import date
import numpy as np
from numpy import random
from random import choices

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

import pyodbc
# have to change this
# cnxn = pyodbc.connect('DSN=DevSQLServer')

from sklearn import linear_model
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
zones = ['JCPL','NYC','MABOS','TX','PSEG','BGE','PEPCO','PECO']
# zones = ['JCPL']

# have to update training

training_last_update = 'Apr05'
forecast_last_update =  str(date.today()).replace('-','_')
record_date =  str(date.today()).replace('-','_')
todays_date = str(date.today()).replace('-','/')
# forecast_last_update = '2019_09_09'
# record_date = '2019_09_09'
# todays_date = '2019/09/09'


In [3]:
stats_table =  pd.DataFrame({'stats':['L_MSE_all','L_R2_all','L_MSE_train','L_R2_train','L_MSE_test','L_R2_test',
                                      'K_MSE_all','K_R2_all','K_MSE_train','K_R2_train','K_MSE_test','K_R2_test',
                                      'S_MSE_ALL','S_R2_all','S_MSE_train','S_R2_train','S_MSE_test','S_R2_test']}).set_index('stats')

L_results_dic = {}
K_results_dic = {}

L_results_F_dic = {}
K_results_F_dic ={}

L_BST_error_dic = {}
K_BST_error_dic = {}
S_BST_error_dic = {}

Lasso_result_all = pd.DataFrame()
Knn_result_all = pd.DataFrame()

stacked_results_F_dic = {}
# L_error_t_dic = {}
# K_error_t_dic = {}

### Lasso

In [4]:
for zone in zones:
    
    # read from the preposessiong file

    newdf = pd.read_csv('C:/Users/yangsh6/Desktop/new_data_oct_22/new/model_feed_' + training_last_update +  '_'+ zone +'.csv')
    newdf.WDate =  pd.to_datetime(newdf.WDate)
    newdf.index = newdf.WDate

    # read the forecast file

    forecast_daily = pd.read_csv('C:/Users/yangsh6/Desktop/new_data_oct_22/new/model_feed_forecast_D_' + zone + '_' + forecast_last_update + '.csv')
    if 'DATA_DATE.1' in forecast_daily:
        forecast_daily = forecast_daily.drop(['DATA_DATE.1'], axis=1)
    forecast_daily['DATA_DATE'] = pd.to_datetime(forecast_daily['DATA_DATE'])
    forecast_daily.index = forecast_daily['DATA_DATE']

    # find the mean and sdtdev of all the columns

    stats = pd.DataFrame(np.std(newdf,axis=0))
    stats['mean']=np.mean(newdf,axis=0)
    stats.rename(columns = {0:'std'},inplace=True)

    # standerdize the training data

    from sklearn.preprocessing import StandardScaler
    newdf.fillna(method ='pad', inplace=True)
    newdf.fillna(method ='bfill', inplace=True)
    # newdf.dropna(how='any', inplace=True)
    newdf[['HI_TEMP', 'LOW_TEMP_F','DFN_HI_TEMP_F','DFN_LOW_TEMP_F','MAX_HEAT_IDX_TEMP_F','MIN_HEAT_IDX_TEMP_F',\
           'MIN_WIND_CHL_TEMP_F','MAX_SUST_WIND_SPEED_MPH','MAX_GUST_MPH','PCPN_POSS_PERCENT',\
         'PCPN_AMT_INCH', 'NORM_PCPN_AMT_INCH', 'SNWFL_AMT_INCH','NORM_SNWFL_AMT_INCH', 'EXTM_WIND_SPEED_MPH',\
          'ICE_AMOUNT_INCH',  'HenryHubPriceMMBtu','crude_oil']] = StandardScaler().fit_transform(newdf[['HI_TEMP', 'LOW_TEMP_F','DFN_HI_TEMP_F','DFN_LOW_TEMP_F','MAX_HEAT_IDX_TEMP_F','MIN_HEAT_IDX_TEMP_F',\
           'MIN_WIND_CHL_TEMP_F','MAX_SUST_WIND_SPEED_MPH','MAX_GUST_MPH','PCPN_POSS_PERCENT',\
         'PCPN_AMT_INCH', 'NORM_PCPN_AMT_INCH', 'SNWFL_AMT_INCH','NORM_SNWFL_AMT_INCH', 'EXTM_WIND_SPEED_MPH',\
          'ICE_AMOUNT_INCH',  'HenryHubPriceMMBtu','crude_oil']])

    # standardize the forecast data

    cols = ['HI_TEMP', 'LOW_TEMP_F', 'DFN_HI_TEMP_F',
           'DFN_LOW_TEMP_F', 'MAX_HEAT_IDX_TEMP_F', 'MIN_HEAT_IDX_TEMP_F',
           'MIN_WIND_CHL_TEMP_F', 'MAX_SUST_WIND_SPEED_MPH', 'MAX_GUST_MPH',
           'PCPN_POSS_PERCENT', 'PCPN_AMT_INCH', 'NORM_PCPN_AMT_INCH',
           'SNWFL_AMT_INCH', 'NORM_SNWFL_AMT_INCH', 'EXTM_WIND_SPEED_MPH',
            'HenryHubPriceMMBtu','crude_oil']

    for i in cols:
        forecast_daily[i] = (forecast_daily[i] - stats['mean'][i])/stats['std'][i]


    # add month dummies to daily data


    forecast_daily['month'] = forecast_daily.index.month
    forecast_daily = pd.get_dummies(forecast_daily, prefix='M', columns=['month'])

    one_year = {1,2,3,4,5,6,7,8,9,10,11,12}  
    months_not_in_forecast = one_year - set(forecast_daily.index.month)

    for i in months_not_in_forecast:

        forecast_daily['M_'+ str(i)] = 0


    forecast_daily = forecast_daily.drop(['M_12','DATA_DATE'] , axis=1)

    # remove outliers

    Max = newdf[newdf.index.year > 2016 ].RealTimePriceMwH.max()
    newdf.RealTimePriceMwH[newdf.RealTimePriceMwH > Max] = Max

    # drop the lags in the trainning dataset,because we dont have daily data for the forecast
    forecast_nolags = newdf

    # add ploynomial terms
    # order = 2

    poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)

    forecast_nolags = forecast_nolags.drop(['WDate'],axis=1)

    #train
    train = forecast_nolags[forecast_nolags.index.year < 2018]

    y_train = train['RealTimePriceMwH'].values.reshape(-1,1)
    x_train = poly.fit_transform(train.drop(['RealTimePriceMwH'],axis=1))

    #test
    test = forecast_nolags[forecast_nolags.index.year >= 2018]

    y_test = test['RealTimePriceMwH'].values.reshape(-1,1)
    x_test = poly.fit_transform(test.drop(['RealTimePriceMwH'],axis=1))

    # put all the data in the training
    train_all = forecast_nolags
    y_train_all = train_all['RealTimePriceMwH'].values.reshape(-1,1)
    x_train_all = poly.fit_transform(train_all.drop(['RealTimePriceMwH'],axis=1))


    forecast_daily = forecast_daily[train.drop(['RealTimePriceMwH'],axis=1).columns]


    # lasso regression 

    alphas = 10**np.linspace(10,-2,100)*0.5                                           # define grid

    lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = False)   # define cv
    lassocv.fit(x_train_all,y_train_all)                                              # fit cv
    lasso_all = Lasso(max_iter = 10000, normalize = False)                            # define model for all the data
    lasso_train = Lasso(max_iter = 10000, normalize = False)                          # define model for only training data      
    
    lasso_all.set_params(alpha=lassocv.alpha_)                                        # set params for all model
    lasso_train.set_params(alpha=lassocv.alpha_)                                      # set params for trining model

    lasso_all.fit(x_train_all,y_train_all)                                            # fit all model with all the data
    lasso_train.fit(x_train,y_train)                                                  # fit training model with training data

    Lasso_result = lasso_train.predict(x_train_all).tolist()
    Lasso_result = pd.DataFrame(data = Lasso_result, columns = ['result'])
    Lasso_result['zone'] = zone
    Lasso_result['price'] = train_all['RealTimePriceMwH'].values.tolist()
    Lasso_result_all = Lasso_result_all.append(Lasso_result)

    stats_table[zone] = [mean_squared_error(y_train_all, lasso_all.predict(x_train_all)),
                         r2_score(y_train_all, lasso_all.predict(x_train_all)),
                         mean_squared_error(y_train, lasso_train.predict(x_train)),
                         r2_score(y_train, lasso_train.predict(x_train)),
                         mean_squared_error(y_test, lasso_train.predict(x_test)),
                         r2_score(y_test, lasso_train.predict(x_test)),
                         0,0,0,0,0,0,
                         0,0,0,0,0,0]    
                                                                                      # write the model stats 
    
    ### generate the predictions

    results_d = pd.DataFrame(lasso_all.predict(poly.fit_transform(forecast_daily)) ,columns=['Pred'])
#     Lasso_f = lasso_all.predict(poly.fit_transform(forecast_daily))

    results_d.index = forecast_daily.index
    
    L_results_dic["{}".format(zone)] = results_d.groupby(results_d.index.month).Pred.mean()
    L_results_F_dic["{}".format(zone)] = results_d
    
    ### Generate 12 month forecast
    ### first 6 months are from the error distribution
    ### the rest of the months are from bootsrap
    ### the following code only generates the bst+pred error, i.e. the predicted monthes
    
    error = train_all['RealTimePriceMwH']-lasso_train.predict(x_train_all).tolist()    

    error = pd.DataFrame(error)
    error['M'] = error.index.month



#     error_t = test['RealTimePriceMwH']-lasso_train.predict(x_test).tolist()
#     error_t = pd.DataFrame(error_t)
#     error_t['M'] = error_t.index.month
    
#     L_error_t_dic["{}".format(zone)] = error_t
    
    monthly_error_result = pd.DataFrame()

    
    ### below is the bootstrap loop for sampling monthly mean from the errors
    for i in range(1,13) :

        sample = error[error.M == i].RealTimePriceMwH.values #RealTimePriceMwH is the errors
        means = []

        for j in range(1,1001):
#             means.append(np.nanmean([random.choice(sample) for _ in range(1,30)]))
            means.append(np.nanmean(choices(sample,k=30))) 
                                                               ### both of them work

        monthly_error_result[i] = means  # monthly_error_result is that df with all the errors 12 month*1000 samples
        
    bootsrap_error = monthly_error_result[list(set(results_d.index.month))]

    for i in bootsrap_error:
        bootsrap_error[i] += results_d.groupby(results_d.index.month).mean().Pred[i]

    rename_months = {1: 'Jan20',2: 'Feb20',3: 'Mar19',4: 'Apr19',5: 'May19',6: 'Jun19',
             7: 'Jul19',8: 'Aug19',9: 'Sep19',10: 'Oct19',11: 'Nov19',12: 'Dec19'} 

    bootsrap_error.columns = bootsrap_error.columns.to_series().map(rename_months)
    
    bootsrap_error = bootsrap_error.unstack().reset_index().drop(['level_1'], axis=1).rename(index=str
                                                        ,columns={"level_0": "Contract_m",0:"Price"})
                                                                           # save the data as database format
    
    bootsrap_error['Zone']= zone
    bootsrap_error['Method']= 'PI_Pred'
    bootsrap_error['Record_date']= todays_date

    L_BST_error_dic["{}".format(zone)] = bootsrap_error

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yangsh6\

### KNN

In [5]:
for zone in zones:
    
    newdf = pd.read_csv('C:/Users/yangsh6/Desktop/new_data_oct_22/new/model_feed_' + training_last_update +  '_'+ zone +'.csv')
    newdf.WDate =  pd.to_datetime(newdf.WDate)
    newdf = newdf.set_index('WDate')
    newdf.fillna(method ='pad', inplace=True)
    newdf.fillna(method ='bfill', inplace=True)

    Max = newdf[newdf.index.year > 2016 ].RealTimePriceMwH.max()
    newdf.RealTimePriceMwH[newdf.RealTimePriceMwH > Max] = Max

    forecast_nolags = newdf



    # read the forecast file

    forecast_daily = pd.read_csv('C:/Users/yangsh6/Desktop/new_data_oct_22/new/model_feed_forecast_D_' + zone + '_' + forecast_last_update + '.csv')
    if 'DATA_DATE.1' in forecast_daily:
        forecast_daily = forecast_daily.drop(['DATA_DATE.1'], axis=1)
    forecast_daily['DATA_DATE'] = pd.to_datetime(forecast_daily['DATA_DATE'])


    #train
    train = forecast_nolags[forecast_nolags.index.year < 2018]

    y_train = train['RealTimePriceMwH'].values.reshape(-1,1)
    x_train = train.drop(['RealTimePriceMwH'],axis=1)
    # #test
    test = forecast_nolags[forecast_nolags.index.year >= 2018]

    y_test = test['RealTimePriceMwH'].values.reshape(-1,1)
    x_test = test.drop(['RealTimePriceMwH'],axis=1)

    #train all
    y_train_knn_all = forecast_nolags['RealTimePriceMwH'].values.reshape(-1,1)
    x_train_knn_all = forecast_nolags.drop(['RealTimePriceMwH'],axis=1)

    y_train_knn = train['RealTimePriceMwH'].values.reshape(-1,1)
    x_train_knn = train.drop(['RealTimePriceMwH'],axis=1)

    # #test knn

    y_test_knn = test['RealTimePriceMwH'].values.reshape(-1,1)
    x_test_knn = test.drop(['RealTimePriceMwH'],axis=1)


    scaler = MinMaxScaler()

    X_all_scaled = scaler.fit_transform(x_train_knn_all)

    X_train_scaled = scaler.transform(x_train_knn)

    X_test_scaled = scaler.transform(x_test_knn)

    param_grid = {'n_neighbors':range(1,20)}
    knn_grid_all = GridSearchCV(neighbors.KNeighborsRegressor(), cv = 5, param_grid = param_grid, scoring = 'neg_mean_absolute_error' )
    knn_grid_train = GridSearchCV(neighbors.KNeighborsRegressor(), cv = 5, param_grid = param_grid, scoring = 'neg_mean_absolute_error' )

    knn_grid_all.fit(X_all_scaled, y_train_knn_all)
    knn_grid_train.fit(X_train_scaled, y_train_knn)
    
    Knn_result = [item for sublist in knn_grid_all.predict(X_all_scaled).tolist() for item in sublist]
    Knn_result = pd.DataFrame(data = Knn_result, columns = ['result'])
    Knn_result['zone'] = zone
    Knn_result_all = Knn_result_all.append(Knn_result)

    stats_table[zone][6:12] = [mean_squared_error(y_train_knn_all,knn_grid_all.predict(X_all_scaled)),
                             r2_score(y_train_knn_all, knn_grid_all.predict(X_all_scaled)),
                             mean_squared_error(y_train_knn, knn_grid_train.predict(X_train_scaled)),
                             r2_score(y_train_knn, knn_grid_train.predict(X_train_scaled)),
                             mean_squared_error(y_test_knn, knn_grid_train.predict(X_test_scaled)),
                             r2_score(y_test_knn, knn_grid_train.predict(X_test_scaled))]  

    # add month dummies to daily data
    forecast_daily = forecast_daily.set_index('DATA_DATE')
    forecast_daily['month'] = forecast_daily.index.month
    forecast_daily = pd.get_dummies(forecast_daily, prefix='M', columns=['month'])

    one_year = {1,2,3,4,5,6,7,8,9,10,11,12}  
    months_not_in_forecast = one_year - set(forecast_daily.index.month)

    for i in months_not_in_forecast:

        forecast_daily['M_'+ str(i)] = 0


    forecast_daily_o = forecast_daily.drop(['M_12'] , axis=1)
    forecast_daily_o = forecast_daily_o[x_train_knn_all.columns]
    forecast_daily_test = scaler.transform(forecast_daily_o)
    forecast_daily = pd.DataFrame(data = forecast_daily_test, columns = forecast_daily_o.columns)

    results_d = pd.DataFrame(knn_grid_all.predict(forecast_daily) ,columns=['Pred'])
#     Knn_f = knn_grid_all.predict(forecast_daily)

    results_d.index = forecast_daily_o.index
    

    K_results_dic["{}".format(zone)] = results_d.groupby(results_d.index.month).Pred.mean()
    K_results_F_dic["{}".format(zone)] = results_d
    
    
    
    ### Generate 12 month forecast
    ### first 6 months are from the error distribution
    ### the rest of the months are from bootsrap
    ### the following code only generates the bst+pred error, i.e. the predicted monthes

    error = y_train_knn_all - knn_grid_all.predict(X_all_scaled)
    error = pd.DataFrame(error,columns=['error'])
    error.index = forecast_nolags.index
    error['M'] = error.index.month

    
#     error_t = y_test-knn_grid_train.predict(X_test_scaled)
#     error_t = pd.DataFrame(error_t,columns=['error'])
#     error_t.index = test.index
#     error_t['M'] = error_t.index.month
    
#     K_error_t_dic["{}".format(zone)] = error_t
    
    
    
    monthly_error_result = pd.DataFrame()


    ### below is the bootstrap loop for sampling monthly mean from the errors
    for i in range(1,13) :

        sample = error[error.M == i].error.values
        means = []

        for j in range(1,1001):
    #             means.append(np.nanmean([random.choice(sample) for _ in range(1,30)]))
            means.append(np.nanmean(choices(sample,k=30))) 
                                                               ### both of them work

        monthly_error_result[i] = means

    bootsrap_error = monthly_error_result[list(set(results_d.index.month))]

    for i in bootsrap_error:
        bootsrap_error[i] += results_d.groupby(results_d.index.month).mean().Pred[i]

    rename_months = {1: 'Jan20',2: 'Feb20',3: 'Mar19',4: 'Apr19',5: 'May19',6: 'Jun19',
             7: 'Jul19',8: 'Aug19',9: 'Sep19',10: 'Oct19',11: 'Nov19',12: 'Dec19'} 

    bootsrap_error.columns = bootsrap_error.columns.to_series().map(rename_months)

    bootsrap_error = bootsrap_error.unstack().reset_index().drop(['level_1'], axis=1).rename(index=str
                                                        ,columns={"level_0": "Contract_m",0:"Price"})
                                                                           # save the data as database format

    bootsrap_error['Zone']= zone
    bootsrap_error['Method']= 'PI_Pred_K'
    bootsrap_error['Record_date']= todays_date

    K_BST_error_dic["{}".format(zone)] = bootsrap_error


C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

### Stacked Model

In [6]:
for zone, df in L_results_F_dic.items():

    
    ### Build the models of the zones
    new_df = train_all[['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11']]
    new_df['Lasso_result'] = Lasso_result_all[Lasso_result_all.zone == zone].result.tolist()
    new_df['Knn_result'] = Knn_result_all[Knn_result_all.zone == zone].result.tolist()
    new_df['RealTimePriceMwH'] = Lasso_result_all[Lasso_result_all.zone == zone].price.tolist()
    
    for month in new_df[['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11']]:
        for model in new_df[['Lasso_result','Knn_result']]:
            name = month + '_' + model
            new_df[name] = new_df[month] * new_df[model]
        
    #train
    train = new_df[new_df.index.year < 2018]

    y_train = train['RealTimePriceMwH'].values.reshape(-1,1)
    x_train = train.drop(['RealTimePriceMwH'],axis=1)

    # #test
    test = new_df[new_df.index.year >= 2018]

    y_test = test['RealTimePriceMwH'].values.reshape(-1,1)
    x_test = test.drop(['RealTimePriceMwH'],axis=1)

    #train all
    y_train_stacked_all = new_df['RealTimePriceMwH'].values.reshape(-1,1)
    x_train_stacked_all = new_df.drop(['RealTimePriceMwH'],axis=1)
    
    model = sm.OLS(y_train,sm.add_constant(x_train)).fit()
    
    model_all = sm.OLS(y_train_stacked_all,sm.add_constant(x_train_stacked_all)).fit()
    
    
   # this is the forecast
    
    df_s = df
    df_s = df_s.rename(columns = {"Pred": "Lasso_result"})
    df_s['Knn_result'] =  K_results_F_dic[zone].Pred
    
    df_s['month'] = df_s.index.month
    df_s = pd.get_dummies(df_s, prefix='M', columns=['month'])
    for i in months_not_in_forecast:
        df_s['M_'+ str(i)] = 0
    df_s = df_s.drop(['M_12'] , axis=1)
    
    for month in df_s[['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11']].columns:
        for model_result in df_s[['Lasso_result','Knn_result']].columns:
            name = month + '_' + model_result
            df_s[name] = df_s[month] * df_s[model_result]

    df_s = df_s[x_train_stacked_all.columns] 
### save the stacked results in a dictionary
    df_s['stacked_result']= model_all.predict(sm.add_constant(df_s)) 
    
    
    
    stacked_results_F_dic["{}".format(zone)] = df_s
    
    ### get them stats table
    stats_table[zone][12:] = [mean_squared_error(y_train_stacked_all,model_all.predict(sm.add_constant(x_train_stacked_all))),
                             r2_score(y_train_stacked_all,model_all.predict(sm.add_constant(x_train_stacked_all))),
                             mean_squared_error(y_train, model.predict(sm.add_constant(x_train))),
                             r2_score(y_train, model.predict(sm.add_constant(x_train))),
                             mean_squared_error(y_test, model.predict(sm.add_constant(x_test))),
                             r2_score(y_test, model.predict(sm.add_constant(x_test)))]  


    error = new_df['RealTimePriceMwH'] - model_all.predict(sm.add_constant(x_train_stacked_all))
    error = pd.DataFrame(error,columns=['error'])
    error.index = forecast_nolags.index
    error['M'] = error.index.month


    monthly_error_result = pd.DataFrame()


    ### below is the bootstrap loop for sampling monthly mean from the errors
    for i in range(1,13) :

        sample = error[error.M == i].error.values
        means = []

        for j in range(1,1001):
    #             means.append(np.nanmean([random.choice(sample) for _ in range(1,30)]))
            means.append(np.nanmean(choices(sample,k=30))) 
                                                               ### both of them work

        monthly_error_result[i] = means





    bootsrap_error = monthly_error_result[list(set(results_d.index.month))]

    for i in bootsrap_error:
        bootsrap_error[i] += df_s.groupby(df_s.index.month).mean().stacked_result[i]

    rename_months = {1: 'Jan20',2: 'Feb20',3: 'Mar20',4: 'Apr20',5: 'May20',6: 'Jun19',
             7: 'Jul19',8: 'Aug19',9: 'Sep19',10: 'Oct19',11: 'Nov19',12: 'Dec19'} 

    bootsrap_error.columns = bootsrap_error.columns.to_series().map(rename_months)

    bootsrap_error = bootsrap_error.unstack().reset_index().drop(['level_1'], axis=1).rename(index=str
                                                        ,columns={"level_0": "Contract_m",0:"Price"})
                                                                           # save the data as database format

    bootsrap_error['Zone']= zone
    bootsrap_error['Method']= 'stacked'
    bootsrap_error['Record_date']= todays_date
    
    bootsrap_error.columns = ['Contract_M_Pred','Price_Pred','Zone_Pred','Method_Pred','Record_Date_Pred']

    S_BST_error_dic["{}".format(zone)] = bootsrap_error
    

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [7]:
table= pd.DataFrame()
for key, value in S_BST_error_dic.items():
    table = table.append(value)


## drop the months that has less than 10 days

month_keep = set(forecast_daily_o.groupby(forecast_daily_o.index.month).filter(lambda x: x['HI_TEMP'].count() > 10).index.month)
month_keep= [rename_months[k] for k in month_keep]
table = table[table.Contract_M_Pred.isin(month_keep)]
table['Zone_Pred'] = table['Zone_Pred'].replace('NYC','ZoneJ')

In [8]:
stacked_results_F_dic

{'JCPL':             M_1  M_2  M_3  M_4  M_5  M_6  M_7  M_8  M_9  M_10       ...        \
 DATA_DATE                                                           ...         
 2019-10-21    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-22    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-23    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-24    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-25    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-26    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-27    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-28    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-29    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10-30    0    0    0    0    0    0    0    0    0     1       ...         
 2019-10

In [9]:
S_BST_error_dic

{'JCPL':      Contract_M_Pred  Price_Pred Zone_Pred Method_Pred Record_Date_Pred
 0              Jan20   32.380538      JCPL     stacked       2019/10/21
 1              Jan20   38.453595      JCPL     stacked       2019/10/21
 2              Jan20   39.292906      JCPL     stacked       2019/10/21
 3              Jan20   38.616547      JCPL     stacked       2019/10/21
 4              Jan20   31.083797      JCPL     stacked       2019/10/21
 5              Jan20   29.211849      JCPL     stacked       2019/10/21
 6              Jan20   37.421819      JCPL     stacked       2019/10/21
 7              Jan20   40.168999      JCPL     stacked       2019/10/21
 8              Jan20   39.865139      JCPL     stacked       2019/10/21
 9              Jan20   30.611859      JCPL     stacked       2019/10/21
 10             Jan20   34.196181      JCPL     stacked       2019/10/21
 11             Jan20   35.539339      JCPL     stacked       2019/10/21
 12             Jan20   36.651330      JCPL

In [10]:
table

,Contract_M_Pred,Price_Pred,Zone_Pred,Method_Pred,Record_Date_Pred
0,Jan20,32.380538,JCPL,stacked,2019/10/21
1,Jan20,38.453595,JCPL,stacked,2019/10/21
2,Jan20,39.292906,JCPL,stacked,2019/10/21
3,Jan20,38.616547,JCPL,stacked,2019/10/21
4,Jan20,31.083797,JCPL,stacked,2019/10/21
5,Jan20,29.211849,JCPL,stacked,2019/10/21
6,Jan20,37.421819,JCPL,stacked,2019/10/21
7,Jan20,40.168999,JCPL,stacked,2019/10/21
8,Jan20,39.865139,JCPL,stacked,2019/10/21
9,Jan20,30.611859,JCPL,stacked,2019/10/21


In [11]:
table.groupby(['Zone_Pred','Contract_M_Pred']).Price_Pred.mean()

Zone_Pred  Contract_M_Pred
BGE        Apr20              35.963880
           Dec19              33.594380
           Feb20              28.860325
           Jan20              20.670510
           Mar20              34.147814
           Nov19              32.251689
           Oct19              34.229907
JCPL       Apr20              26.576506
           Dec19              26.826342
           Feb20              26.470985
           Jan20              34.010679
           Mar20              27.746494
           Nov19              27.179128
           Oct19              23.313542
MABOS      Apr20              37.660808
           Dec19              45.683751
           Feb20              66.234967
           Jan20              60.579303
           Mar20              39.519187
           Nov19              34.430633
           Oct19              35.896931
PECO       Apr20              31.116582
           Dec19              24.820035
           Feb20              28.459803
           Ja

In [12]:
stats_table

,JCPL,NYC,MABOS,TX,PSEG,BGE,PEPCO,PECO
stats,,,,,,,,
L_MSE_all,202.338806,283.230577,468.721741,164.930379,184.223600,230.162192,263.077261,236.431560
L_R2_all,0.734281,0.760026,0.702978,0.205633,0.764405,0.671818,0.616419,0.662985
L_MSE_train,200.308260,290.679818,450.837194,115.207786,171.145420,235.150207,253.598472,216.582621
L_R2_train,0.738175,0.769548,0.736281,0.295465,0.785287,0.665693,0.632057,0.693648
L_MSE_test,238.483093,288.506411,645.056120,384.081865,297.447208,232.288763,336.813146,434.354621
L_R2_test,0.680431,0.662213,0.385795,-0.009446,0.585184,0.661423,0.496996,0.360416
K_MSE_all,163.613405,296.777158,509.303196,165.496514,217.556352,296.994143,249.068741,219.239463
K_R2_all,0.785137,0.748548,0.677262,0.202906,0.721777,0.576525,0.636844,0.687491
K_MSE_train,184.051640,273.327479,522.266237,117.139821,169.599484,308.172985,313.466718,250.163163


In [13]:
from sqlalchemy import create_engine, event
engine = create_engine("mssql+pyodbc://:@PROD_NEW")

In [14]:
to_push = table
to_push.astype('str').to_sql('EP_Prediction_PI_archive', engine, if_exists='append', chunksize=50, index= None, schema='SCMAnalyticsDev.dbo')

In [15]:
## change this variable every time before running the report
file_date =str(date.today()).replace('-','')

In [16]:
to_push.to_csv('C:/Users/yangsh6/Desktop/new_data_oct_22/Pred+PI_archive/Pred_'+  file_date +'.csv')

In [52]:
# interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
# df_inter = interaction.fit_transform(new_df)

In [67]:
# for month in new_df.columns[0:11]:
#     for model in new_df.columns[11:13]:
#         name = month + '_' + model
#         new_df[name] = new_df[month] * new_df[model]

C:\Users\yangsh6\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [70]:
# regression = LinearRegression(normalize=True)
# model_inter = sm.OLS(y_train_all,sm.add_constant(new_df)).fit()
# predictions = model_inter.predict(sm.add_constant(new_df))
# mean_squared_error(y_train_all, predictions)

# model_inter.summary()

In [ ]:
# for zone in zones:

#     new_df = train_all[['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11']]
#     new_df['Lasso_result'] = Lasso_result_all[Lasso_result_all.zone == zone].result.tolist()
#     new_df['Knn_result'] = Knn_result_all[Knn_result_all.zone == zone].result.tolist()
#     new_df['RealTimePriceMwH'] = Lasso_result_all[Lasso_result_all.zone == zone].price.tolist()
    
#     for month in new_df[['M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'M_10', 'M_11']]:
#         for model in new_df[['Lasso_result','Knn_result']]:
#             name = month + '_' + model
#             new_df[name] = new_df[month] * new_df[model]
        
#     #train
#     train = new_df[new_df.index.year < 2018]

#     y_train = train['RealTimePriceMwH'].values.reshape(-1,1)
#     x_train = train.drop(['RealTimePriceMwH'],axis=1)

#     # #test
#     test = new_df[new_df.index.year >= 2018]

#     y_test = test['RealTimePriceMwH'].values.reshape(-1,1)
#     x_test = test.drop(['RealTimePriceMwH'],axis=1)

#     #train all
#     y_train_stacked_all = new_df['RealTimePriceMwH'].values.reshape(-1,1)
#     x_train_stacked_all = new_df.drop(['RealTimePriceMwH'],axis=1)
    
#     model = sm.OLS(y_train,sm.add_constant(x_train)).fit()
    
#     model_all = sm.OLS(y_train_stacked_all,sm.add_constant(x_train_stacked_all)).fit()
    
    
#     stats_table[zone][12:] = [mean_squared_error(y_train_stacked_all,model_all.predict(sm.add_constant(x_train_stacked_all))),
#                              r2_score(y_train_stacked_all,model_all.predict(sm.add_constant(x_train_stacked_all))),
#                              mean_squared_error(y_train, model.predict(sm.add_constant(x_train))),
#                              r2_score(y_train, model.predict(sm.add_constant(x_train))),
#                              mean_squared_error(y_test, model.predict(sm.add_constant(x_test))),
#                              r2_score(y_test, model.predict(sm.add_constant(x_test)))]  



In [ ]:
# error = y_train_stacked_all - model_all.predict(sm.add_constant(x_train_stacked_all))
# error = pd.DataFrame(error,columns=['error'])
# error.index = forecast_nolags.index
# error['M'] = error.index.month


# monthly_error_result = pd.DataFrame()


# ### below is the bootstrap loop for sampling monthly mean from the errors
# for i in range(1,13) :

#     sample = error[error.M == i].error.values
#     means = []

#     for j in range(1,1001):
# #             means.append(np.nanmean([random.choice(sample) for _ in range(1,30)]))
#         means.append(np.nanmean(choices(sample,k=30))) 
#                                                            ### both of them work

#     monthly_error_result[i] = means
    
    
    
    

# bootsrap_error = monthly_error_result[list(set(results_d.index.month))]

# # for i in bootsrap_error:
# #     bootsrap_error[i] += results_d.groupby(results_d.index.month).mean().Pred[i]

# rename_months = {1: 'Jan20',2: 'Feb20',3: 'Mar19',4: 'Apr19',5: 'May19',6: 'Jun19',
#          7: 'Jul19',8: 'Aug19',9: 'Sep19',10: 'Oct19',11: 'Nov19',12: 'Dec19'} 

# bootsrap_error.columns = bootsrap_error.columns.to_series().map(rename_months)

# bootsrap_error = bootsrap_error.unstack().reset_index().drop(['level_1'], axis=1).rename(index=str
#                                                     ,columns={"level_0": "Contract_m",0:"Price"})
#                                                                        # save the data as database format

# bootsrap_error['Zone']= zone
# bootsrap_error['Method']= 'PI_Pred_K'
# bootsrap_error['Record_date']= todays_date

# K_BST_error_dic["{}".format(zone)] = bootsrap_error